# Ingeniería de Características y Selección Basada en Correlación

En todo proceso de modelado supervisado, la calidad de las variables de entrada puede llegar a ser tan determinante como el propio algoritmo. En este notebook proponemos una estrategia estadística para segmentar nuestro espacio de características en dos grupos: uno con las **columnas más fuertemente correlacionadas** y otro con aquellas de **correlación moderada** con respecto a la variable objetivo. De este modo, podremos entrenar modelos con distintos niveles de información y comparar directamente su desempeño.

Primero, ampliamos nuestro set original de datos generando nuevas variables —por ejemplo, interacciones o polinomios— que potencialmente capten patrones no lineales. A continuación, medimos la relevancia de cada característica mediante el coeficiente de correlación de Pearson:

$$
r_i = \frac{\mathrm{cov}(X_i, Y)}{\sigma_{X_i}\,\sigma_{Y}}
$$

y cuantificamos la redundancia usando el **Factor de Inflación de la Varianza (VIF)**:

$$
\mathrm{VIF}_i = \frac{1}{1 - R_i^2}
$$

donde $R_i^2$ es el coeficiente de determinación al ajustar $X_i$ como variable dependiente frente al resto de columnas del dataset.

Para decidir qué columnas pasan a cada grupo, estudiamos la **distribución empírica** de los valores $|r_i|$. Definimos dos umbrales basados en percentiles:

- El **66.6 %** señala el corte para nuestra **Tabla “Relevante”** (correlaciones más altas).  
- El **33.3 %** marca el límite mínimo para la **Tabla “Semi-Relevante”** (correlaciones moderadas).

El resto de las variables, con $|r_i|$ por debajo del percentil 33.3, queda fuera del conjunto de entrenamiento por aportar muy poca señal.

En la siguiente sección presentamos una visualización con una curva normal de referencia:

![Distribución de correlaciones con la variable objetivo](Graphics\Ej_filtrado_correlacion.png)

Las áreas sombreadas a la derecha de cada umbral permiten apreciar de un solo vistazo la proporción de columnas seleccionadas.

Finalmente, crearemos dos DataFrames —uno con las columnas del grupo “Relevante” y otro con las del grupo “Semi-Relevante”— y aplicaremos la **ecuación de pronóstico** proporcionada por el concurso para predecir el número de unidades vendidas en los 16 días siguientes. Evaluaremos cada modelo comparando métricas de error de regresión como **MAE** (Error Absoluto Medio) y **RMSE** (Raíz del Error Cuadrático Medio) para determinar si el uso de un conjunto de variables más amplio (Semi-Relevante) mejora o empeora las predicciones frente al conjunto más restringido (Relevante). Esta metodología aporta rigor estadístico y flexibilidad para explorar el trade-off entre complejidad del modelo y precisión de la predicción.


Claro, aquí tienes una versión mejorada, más clara, coherente y profesional del texto, junto con la fórmula de variación porcentual diaria y un ejemplo más estructurado del análisis de desfase:

---

Vamos a utilizar como plantilla base la tabla `train`, a partir de la cual crearemos dos nuevas copias denominadas `Test_50_insight` y `Test_100_insight`.

Planeo trabajar con el DataFrame `oil`, del cual extraeremos las columnas `date` y `dcoilwtico` (precio del petróleo WTI). Con esta información, calcularemos la **variación porcentual diaria del precio del petróleo** y la almacenaremos en una nueva columna llamada `oil_price`.

Esta columna será posteriormente combinada con la tabla `train`, alineando los valores por la columna `date`, para así conocer cómo varió el precio del petróleo cada día específico.

### Cálculo de la variación porcentual diaria

La fórmula para calcular la variación porcentual diaria es:

$$
\text{variación}_{t} = \frac{P_{t} - P_{t-1}}{P_{t-1}} \times 100
$$

Donde:

* $P_t$ es el precio del petróleo en el día actual
* $P_{t-1}$ es el precio del día anterior

### Justificación del desfase temporal

Inicialmente se pensó en **adelantar un día** la variable `oil_price`, ya que el precio del petróleo actual podría influir en las decisiones del mercado **al día siguiente**, no en el mismo día en que se registra el precio. Sin embargo, para tomar una decisión informada, se propone realizar un análisis de **correlación** entre la variable `oil_price` y otras variables objetivo, considerando tres escenarios distintos:

1. **Desfase -1 (precio del día anterior):**
   Se utiliza el precio del petróleo del día anterior para predecir el comportamiento del día actual.
2. **Sin desfase (precio actual):**
   Se utiliza el precio del petróleo del mismo día.
3. **Desfase +1 (precio del día siguiente):**
   Se usa el precio del día siguiente, bajo la hipótesis de que el precio de hoy refleja la reacción al comportamiento del día anterior.

### Ejemplo ilustrativo de desfases

Aquí tienes un ejemplo más claro de cómo funciona el desfase en los precios del petróleo. Supongamos que los precios diarios son los siguientes:

| Fecha      | Precio WTI |
| ---------- | ---------- |
| 17 de mayo | 1.01       |
| 18 de mayo | 1.05       |
| 19 de mayo | 1.02       |

Y que nuestra plantilla `train` contiene las mismas fechas como índice para unir:

| Fecha      | … otras columnas … |
| ---------- | ------------------ |
| 17 de mayo | …                  |
| 18 de mayo | …                  |
| 19 de mayo | …                  |

Ahora veamos los tres escenarios:

---

### 1. Desfase -1 (precio del día anterior)

Aquí asignamos al 18 de mayo el precio del 17 de mayo, y al 19 de mayo el del 18 de mayo:

| Fecha (train) | oil\_price (t-1) |
| ------------- | ---------------- |
| 17 de mayo    | —                |
| 18 de mayo    | 1.01             |
| 19 de mayo    | 1.05             |

* **Interpretación**: Para el 18 usas el precio del 17, asumiendo que el mercado reacciona con un día de retraso.

---

### 2. Sin desfase (precio del mismo día)

Asignamos a cada fecha su propio precio:

| Fecha (train) | oil\_price (t) |
| ------------- | -------------- |
| 17 de mayo    | 1.01           |
| 18 de mayo    | 1.05           |
| 19 de mayo    | 1.02           |

* **Interpretación**: El modelo ve el precio del petróleo del mismo día.

---

### 3. Desfase +1 (precio del día siguiente)

Aquí adelantamos un día: al 17 de mayo le ponemos el precio del 18, y al 18 el del 19:

| Fecha (train) | oil\_price (t+1) |
| ------------- | ---------------- |
| 17 de mayo    | 1.05             |
| 18 de mayo    | 1.02             |
| 19 de mayo    | —                |

* **Interpretación**: El modelo “ve” el precio que estará disponible mañana, ideal si crees que las señales del petróleo ya anticipan movimientos un día antes.

---

Con estos tres conjuntos de datos podrás calcular el coeficiente de correlación de cada versión de `oil_price` con tu variable objetivo y decidir cuál desfase aporta más valor predictivo.


## `HOLIDAYS_Events`

Este es otro *dataframe* que contiene columnas con información útil que podemos aprovechar.  
La idea es crear dos nuevas columnas en nuestro conjunto de datos `train` (y también `test`):

1. **`es_festivo`**: una columna booleana que indicará si la fecha corresponde a un día festivo (`True` o `False`).
2. **`tipo_festivo`**: una columna categórica que especificará el tipo de día festivo. Para facilitar su uso en modelos de IA (que no aceptan variables de tipo `string` directamente), se codificará de forma numérica:

   - `0`: festivo local  
   - `1`: festivo regional  
   - `2`: festivo nacional

> ⚠️ **Nota importante:**  
Para clasificar correctamente si un día es festivo y su tipo, se debe tener en cuenta la información contenida en `stores.csv`.  
Cada fila de los dataframes `train` y `test` contiene una columna llamada `store_nbr`, que representa el número de tienda.  
Mediante este número, se puede localizar la tienda correspondiente en `stores.csv`, que contiene datos clave como la ciudad, estado o región de cada tienda.  
Posteriormente, al cruzar esa ubicación con la fecha correspondiente en el dataframe `HOLIDAYS_Events`, es posible determinar:

- Si ese día es festivo o no para esa tienda (`es_festivo`)
- Y en caso afirmativo, el tipo de festivo (`tipo_festivo`: local, regional o nacional)

Estas nuevas columnas nos permitirán enriquecer el análisis y posiblemente mejorar el rendimiento del modelo.


Del DataFrame `items` extraeremos las columnas `family`, `class` y `perishable` con el objetivo de convertirlas a variables numéricas. En particular, la columna `perishable` se transformará en una variable binaria, donde:

- `1` indicará que el producto es perecedero,
- `0` que no lo es.

Por otro lado, del DataFrame `transactions` extraeremos la columna `transactions`, ya que la utilizaremos como variable explicativa en el análisis.


## Estructura Final del DataFrame Integrado

A continuación se muestra un ejemplo de cómo quedaría la arquitectura final del DataFrame después de integrar y transformar las distintas fuentes de datos (`sales`, `items`, `transactions`, `holidays_events`, `oil`), dejando todas las columnas en formato numérico:

| id  | date       | store_nbr | item_nbr | unit_sales | onpromotion | is_festive | type_festive | type_local_festive | Price_oil_pct | family_items | class_items | perishable | transactions |
|-----|------------|-----------|----------|------------|-------------|------------|--------------|---------------------|----------------|---------------|-------------|------------|--------------|
| 1   | 2017-08-15 | 1         | 103520   | 3.0        | 1           | 1          | 2            | 1                   | 0.012          | 4             | 1013        | 1          | 1345         |
| 2   | 2017-08-15 | 1         | 105574   | 0.0        | 0           | 1          | 2            | 1                   | 0.012          | 2             | 2020        | 1          | 1345         |
| 3   | 2017-08-16 | 2         | 103520   | 5.0        | 1           | 0          | 0            | 0                   | -0.006         | 4             | 1013        | 1          | 1120         |
| 4   | 2017-08-16 | 2         | 209211   | 8.0        | 0           | 0          | 0            | 0                   | -0.006         | 3             | 3001        | 1          | 1120         |

### Descripción de columnas

- **`store_nbr`**: Identificador numérico de la tienda.
- **`item_nbr`**: Identificador numérico del producto.
- **`unit_sales`**: Unidades vendidas del producto en esa fecha y tienda.
- **`onpromotion`**: Indicador binario (`1` si el producto estaba en promoción, `0` si no).
- **`is_festive`**: Indicador binario (`1` si la fecha es festiva, `0` si no).
- **`type_festive`**: Tipo de festividad codificado numéricamente (ej. `0`: Ninguna, `1`: Evento, `2`: Holiday).
- **`type_local_festive`**: Indicador binario (`1` si la festividad es local, `0` si no).
- **`Price_oil_pct`**: Variación porcentual del precio del petróleo respecto al día anterior.
- **`family_items`**: Categoría general del producto codificada numéricamente (Label Encoding).
- **`class_items`**: Subcategoría o clase numérica del producto.
- **`perishable`**: Indicador binario (`1` si el producto es perecedero, `0` si no).
- **`transactions`**: Número total de transacciones realizadas en la tienda ese día.

Este DataFrame está completamente listo para ser utilizado en modelos de predicción o análisis multivariado, cumpliendo con el requisito de tener únicamente variables numéricas.


In [3]:
import os
import dask.dataframe as dd
import pathlib

# 2. Define los archivos CSV de entrada y la carpeta de salida base
input_files = [
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\transactions.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\test.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\stores.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\items.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\holidays_events.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\processed\clear_train.csv",
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\processed\clear_oil_raw_Dukascopy.csv",
]

base_output_dir = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready"
chunk_size_mb = 64
blocksize = chunk_size_mb * 1024 ** 2

# 3. Procesa cada archivo CSV individualmente
def convert_csv_to_parquet(csv_path):
    file_name = pathlib.Path(csv_path).stem
    output_path = os.path.join(base_output_dir, file_name)
    os.makedirs(output_path, exist_ok=True)

    print(f"Procesando {csv_path} -> {output_path}")

    df = dd.read_csv(
        csv_path,
        blocksize=blocksize,
        assume_missing=True
    )

    print(f"Particiones: {df.npartitions}")

    df.to_parquet(
        output_path,
        write_index=False,
        compression='snappy',
        engine='pyarrow'
    )

    print(f"✅ Guardado en: {output_path}\n")

for csv_file in input_files:
    convert_csv_to_parquet(csv_file)


Procesando D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\transactions.csv -> D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\transactions
Particiones: 1
✅ Guardado en: D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\transactions

Procesando D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\test.csv -> D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\test
Particiones: 1
✅ Guardado en: D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\test

Procesando D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\stores.csv -> D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\stores
Particiones: 1
✅ Guardado en: D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\stores

Procesando D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\items.csv -> D:\Portafolio oficial\Retail Sales Trend Analysis\data

In [ ]:
import dask.dataframe as dd

# ----------------------------
# Rutas de archivos
# ----------------------------
oil_path           = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\clear_oil_raw_Dukascopy"
transactions_path  = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\transactions"
items_path         = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\items"
stores_path        = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\stores"
clear_train_path   = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\clear_train"
holidays_path      = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\holidays_events"
output_path        = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\cleaned_ready\train_with_oil_tx_items_stores_festive.parquet"

# ----------------------------
# 1) Leer y procesar datos de Oil
# ----------------------------
oil_dd = dd.read_parquet(oil_path, engine="pyarrow")
oil_dd = oil_dd.assign(prev_price=oil_dd["dcoilwtico"].shift(1))
oil_dd = (
    oil_dd
    .assign(
        Price_oil_pct=((oil_dd["dcoilwtico"] - oil_dd["prev_price"]) / oil_dd["prev_price"]) * 100
    )
    .fillna({"Price_oil_pct": 0})
    .loc[:, ["date", "Price_oil_pct"]]
)

# ----------------------------
# 2) Leer transacciones
# ----------------------------
tx_dd = (
    dd.read_parquet(transactions_path, engine="pyarrow")
      .astype({"store_nbr": "int32"})
      [["date", "store_nbr", "transactions"]]
)

# ----------------------------
# 3) Leer información de items
# ----------------------------
items_dd = (
    dd.read_parquet(items_path, engine="pyarrow")
      [["item_nbr", "family", "class", "perishable"]]
)

# ----------------------------
# 4) Leer train base
# ----------------------------
train_dd = (
    dd.read_parquet(clear_train_path, engine="pyarrow")
      .astype({"store_nbr": "int32"})
)

# ----------------------------
# 5) Merge Oil
# ----------------------------
train_dd = train_dd.merge(oil_dd, on="date", how="left")

# ----------------------------
# 6) Merge Transacciones y rellenar nulos
# ----------------------------
train_dd = train_dd.merge(
    tx_dd,
    on=["date", "store_nbr"],
    how="left"
)
train_dd = train_dd.assign(
    transactions = train_dd["transactions"].fillna(0).astype("int32")
)

# ----------------------------
# 7) Merge Ítems
# ----------------------------
train_dd = train_dd.merge(items_dd, on="item_nbr", how="left")

# ----------------------------
# 8) Merge Stores (ciudad y estado)
# ----------------------------
stores_dd = (
    dd.read_parquet(stores_path, engine="pyarrow")
      .astype({"store_nbr": "int32"})
      [["store_nbr", "city", "state"]]
)
train_dd = train_dd.merge(
    stores_dd,
    on="store_nbr",
    how="left"
)
train_dd = train_dd.assign(
    city  = train_dd["city"].fillna("Unknown"),
    state = train_dd["state"].fillna("Unknown")
)

# ----------------------------
# 9) Leer holidays_events (incluimos locale_name)
# ----------------------------
hol_dd = (
    dd.read_parquet(holidays_path, engine="pyarrow")
      [["date", "type", "locale", "locale_name", "transferred"]]
      .rename(columns={
          "type": "holiday_type",
          "locale": "holiday_locale",
          "locale_name": "holiday_locale_name"
      })
)

# ----------------------------
# 10) Merge Festivos y crear los flags con la nueva lógica
# ----------------------------
train_dd = train_dd.merge(
    hol_dd,
    on="date",
    how="left"
)

# Normalizamos `transferred` a boolean
train_dd = train_dd.assign(
    transferred = train_dd["transferred"].fillna(False)
)

# definimos el flag is_festive combinando las tres reglas
train_dd = train_dd.assign(
    is_festive=(
        ((train_dd["holiday_locale"] == "National") & (~train_dd["transferred"])) |
        ((train_dd["holiday_locale"] == "Regional") & (train_dd["state"] == train_dd["holiday_locale_name"])) |
        ((train_dd["holiday_locale"] == "Local") &
         (train_dd["city"] == train_dd["holiday_locale_name"]) &
         (~train_dd["transferred"]))
    )
)

# asignamos los valores de tipo y nivel solo donde is_festive==True
train_dd = train_dd.assign(
    type_festive       = train_dd["holiday_type"].where(train_dd["is_festive"], other=None),
    local_type_festive = train_dd["holiday_locale"].where(train_dd["is_festive"], other=None)
)

# ----------------------------
# 10.b) Rellenar valores por defecto y limpiar tipos
# ----------------------------
# Usamos map_partitions para aplicar fillna + infer_objects en cada partición
import pandas as pd

train_dd = train_dd.map_partitions(
    lambda df: (
        df
        .assign(
            is_festive=lambda d: d["is_festive"].fillna(False),
            type_festive=lambda d: d["type_festive"].fillna("Normal Day"),
            local_type_festive=lambda d: d["local_type_festive"].fillna("Sin especificar")
        )
        .infer_objects(copy=False)
    )
)

# ----------------------------
# 10.c) Eliminar las columnas que ya no necesitamos
# ----------------------------
train_dd = train_dd.drop(columns=[
    "holiday_type",
    "holiday_locale",
    "holiday_locale_name",
])

bool_cols = [col for col, dtype in train_dd.dtypes.items() if dtype == 'bool']
train_dd = train_dd.astype({c: 'int8' for c in bool_cols})

# ----------------------------
# 11) Guardar DataFrame final
# ----------------------------
train_dd.to_parquet(
    output_path,
    engine="pyarrow",
    write_index=False
)
# ----------------------------
# 12) Label Encoding de categóricas
# ----------------------------
cat_cols = [
    "family",
    "city",
    "state",
    "type_festive",
    "local_type_festive"
]

# 1) Aseguramos categorías «conocidas» en Dask
train_dd = train_dd.categorize(columns=cat_cols)

# 2) Extraemos los códigos
for col in cat_cols:
    train_dd[col + "_enc"] = train_dd[col].cat.codes

# 3) --- NEW: Construimos y guardamos el diccionario de categorías ANTES de dropear ---
import json
categories_map = {}
for col in cat_cols:
    # `train_dd[col]` todavía existe, así que podemos usar `.cat.categories`
    categories_map[col] = train_dd[col].cat.categories.tolist()

with open(
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\logs\mapeado_train\categories_map.json",
    "w", encoding="utf-8"
) as f:
    json.dump(categories_map, f, ensure_ascii=False, indent=2)

# 4) Ahora sí, eliminamos las columnas originales
train_dd = train_dd.drop(columns=cat_cols)
s
# ----------------------------
# 13) Guardar DataFrame final con codificaciones
# ----------------------------
train_dd.to_parquet(
    output_path,
    engine="pyarrow",
    write_index=False
)
